The following tutorials are available from the [Wallaroo Tutorials Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/notebooks_in_prod).

# Stage 3: Deploy the Model in Wallaroo
 
In this stage, we upload the trained model and the processing steps to Wallaroo, then set up and deploy the inference pipeline. 

Once deployed we can feed the newest batch of data to the pipeline, do the inferences and write the results to a results table.

For clarity in this demo, we have split the training/upload task into two notebooks:

* `02_automated_training_process.ipynb`: Train and pickle ML model.
* `03_deploy_model.ipynb`: Upload the model to Wallaroo and deploy into a pipeline.

Assuming no changes are made to the structure of the model, these two notebooks, or a script based on them, can then be scheduled to run on a regular basis, to refresh the model with more recent training data and update the inference pipeline.

This notebook is expected to run within the Wallaroo instance's Jupyter Hub service to provide access to all required Wallaroo libraries and functionality.

## Resources

The following resources are used as part of this tutorial:

* **data**
  * `data/seattle_housing_col_description.txt`: Describes the columns used as part data analysis.
  * `data/seattle_housing.csv`: Sample data of the Seattle, Washington housing market between 2014 and 2015.
* **code**
  * `simdb.py`: A simulated database to demonstrate sending and receiving queries.
* **models**
  * `housing_model_xgb.onnx`: Model created in Stage 2: Training Process Automation Setup.

## Steps

The process of uploading the model to Wallaroo follows these steps:

* [Connect to Wallaroo](#connect-to-wallaroo): Connect to the Wallaroo instance and set up the workspace.
* [Upload The Model](#upload-the-model): Upload the model and autoconvert for use in the Wallaroo engine.
* [Upload the Processing Modules](#upload-the-processing-modules): Upload the processing modules.
* [Create and Deploy the Pipeline](#create-and-deploy-the-pipeline): Create the pipeline with the model and processing modules as steps, then deploy it.
* [Test the Pipeline](#test-the-pipeline): Verify that the pipeline works with the sample data.

### Connect to Wallaroo

First we import the required libraries to connect to the Wallaroo instance, then connect to the Wallaroo instance.

In [1]:
import json
import pickle
import pandas as pd
import numpy as np
import pyarrow as pa

import simdb # module for the purpose of this demo to simulate pulling data from a database

# from wallaroo.ModelConversion import ConvertXGBoostArgs, ModelConversionSource, ModelConversionInputType
import wallaroo
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import datetime

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

In [3]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

In [4]:
workspace_name = 'housepricing'
model_name = "housepricemodel"
model_file = "./housing_model_xgb.onnx"
pipeline_name = "housing-pipe"

The workspace `housepricing` will either be created or, if already existing, used and set to the current workspace.

In [5]:
new_workspace = get_workspace(workspace_name, wl)
new_workspace

{'name': 'housepricing', 'id': 8, 'archived': False, 'created_by': '5a22be45-71f2-41ae-b79b-ac821205b8f3', 'created_at': '2024-03-06T22:03:26.442508+00:00', 'models': [{'name': 'housepricemodel', 'versions': 4, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 3, 6, 22, 27, 30, 513987, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 3, 6, 22, 3, 50, 983080, tzinfo=tzutc())}, {'name': 'preprocess', 'versions': 3, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 3, 6, 22, 27, 32, 22000, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 3, 6, 22, 12, 1, 367762, tzinfo=tzutc())}, {'name': 'postprocess', 'versions': 3, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 3, 6, 22, 27, 33, 339567, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 3, 6, 22, 12, 2, 446549, tzinfo=tzutc())}], 'pipelines': [{'name': 'housing-pipe', 'create_time': datetime.datetime(2024, 3, 6, 22, 4, 7, 428588, tzinfo=tzutc()), 'definition': '[]'}]}

In [6]:
_ = wl.set_current_workspace(new_workspace)

### Upload The Model

With the connection set and workspace prepared, upload the model created in `02_automated_training_process.ipynb` into the current workspace.

To ensure the model input contract matches the provided input, the configuration `tensor_fields=["tensor"]` is used so regardless of what the model input type is, Wallaroo will ensure inputs of type `tensor` are accepted.

In [7]:
hpmodel = (wl.upload_model(model_name, 
                           model_file, 
                           framework=wallaroo.framework.Framework.ONNX)
                           .configure(tensor_fields=["tensor"]
                                    )
            )

### Create and Deploy the Pipeline

Create the pipeline with the preprocess module, housing model, and postprocess module as pipeline steps, then deploy the newpipeline.

In [8]:
pipeline = wl.build_pipeline(pipeline_name)
# clear if the tutorial was run before
pipeline.clear()

pipeline.add_model_step(hpmodel)

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

name,housing-pipe
created,2024-03-06 22:04:07.428588+00:00
last_updated,2024-03-06 22:30:05.853639+00:00
deployed,True
arch,None
accel,None
tags,
versions,"8ee88c49-3d09-43b6-b3a3-f1fd7010d9ba, 99204f60-a7fc-473c-881e-e17a8d512774, 8322e7d7-d5ce-417e-a626-6dc4ba327b11, 3095d26d-d7f8-4478-ac2f-92df819ceb14, 7b0e4d58-2232-4a32-8757-5152ffb0d6ef, 21952c2b-89ef-45a3-b5c0-1685af58fa10, 1e6ac00f-9e81-48c6-9956-3a5b27e44616, 0f676ed7-9cd3-4021-82d4-8b180aa4fb00, 9729b5fa-2184-4b0c-8c72-906e5871c276, 895dac3a-fbfc-462c-856c-9ad47a946bf2, a4ffd306-70f6-4668-a236-b90522f0fdcb, 374fb9d1-afd1-47bb-a57e-fd6e3134ded2, ab03935a-38e2-41c3-b05b-bb0cde7088c7, 8f2dc40b-f190-4949-9568-021649630d3f, 3af8a7e3-b817-4031-ac68-c6dd09cbad95, b0524141-5a52-4824-b959-d26892699171"
steps,housepricemodel
published,False


### Test the Pipeline

We will use a single query from the simulated `housing_price` table and infer.  When successful, we will undeploy the pipeline to restore the resources back to the Kubernetes environment.

In [9]:
conn = simdb.simulate_db_connection()

# create the query
query = f"select * from {simdb.tablename} limit 1"
print(query)

# read in the data
singleton = pd.read_sql_query(query, conn)
conn.close()

display(singleton.loc[:, ["id", "date", "list_price", "bedrooms", "bathrooms", "sqft_living", "sqft_lot"]])

select * from house_listings limit 1


,id,date,list_price,bedrooms,bathrooms,sqft_living,sqft_lot
0,7129300520,2023-07-24,221900.0,3,1.0,1180,5650


We'll pre-process the data by taking the values above and submitting them into a tensor format.  The helper library `preprocess.py` handles that for us.

In [10]:
# preprocess the data to be uploaded
import preprocess

df_input = pd.DataFrame(preprocess.wallaroo_json(singleton))
df_input

[{'tensor': [3.0, 1.0, 1180.0, 5650.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1180.0, 0.0, 47.5112, -122.257, 1340.0, 5650.0, 69.0, 0.0, 0.0]}]


,tensor
0,"[3.0, 1.0, 1180.0, 5650.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1180.0, 0.0, 47.5112, -122.257, 1340.0, 5650.0, 69.0, 0.0, 0.0]"


In [11]:
result = pipeline.infer(df_input)
result

,time,in.tensor,out.variable,anomaly.count
0,2024-03-06 22:30:23.351,"[3.0, 1.0, 1180.0, 5650.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1180.0, 0.0, 47.5112, -122.257, 1340.0, 5650.0, 69.0, 0.0, 0.0]",[5.3518963],0


With our result, we'll run it through a post process and get the final values.

In [12]:
import postprocess

df_output = pd.DataFrame(postprocess.wallaroo_json(result))
df_output

,out.variable
0,[224852.0]


When finished, we undeploy the pipeline to return the resources back to the environment.

In [13]:
pipeline.undeploy()

name,housing-pipe
created,2024-03-06 22:04:07.428588+00:00
last_updated,2024-03-06 22:30:05.853639+00:00
deployed,False
arch,None
accel,None
tags,
versions,"8ee88c49-3d09-43b6-b3a3-f1fd7010d9ba, 99204f60-a7fc-473c-881e-e17a8d512774, 8322e7d7-d5ce-417e-a626-6dc4ba327b11, 3095d26d-d7f8-4478-ac2f-92df819ceb14, 7b0e4d58-2232-4a32-8757-5152ffb0d6ef, 21952c2b-89ef-45a3-b5c0-1685af58fa10, 1e6ac00f-9e81-48c6-9956-3a5b27e44616, 0f676ed7-9cd3-4021-82d4-8b180aa4fb00, 9729b5fa-2184-4b0c-8c72-906e5871c276, 895dac3a-fbfc-462c-856c-9ad47a946bf2, a4ffd306-70f6-4668-a236-b90522f0fdcb, 374fb9d1-afd1-47bb-a57e-fd6e3134ded2, ab03935a-38e2-41c3-b05b-bb0cde7088c7, 8f2dc40b-f190-4949-9568-021649630d3f, 3af8a7e3-b817-4031-ac68-c6dd09cbad95, b0524141-5a52-4824-b959-d26892699171"
steps,housepricemodel
published,False


With this stage complete, we can proceed to Stage 4: Regular Batch Inference.